In [ ]:
import json
import os
import time
import base64
from google import genai
from google.genai import types
from tqdm.notebook import tqdm

# --- KONFIGURATION AF STIL (SYSTEM PROMPTS) ---

# Stil til Upgrades, Projects og Meta (Firkantede ikoner)
STYLE_ICON = (
    "Windows 95 aesthetic pixel art icon. "
    "16-bit graphics, 256 color VGA palette, dithered shading, low resolution, blocky. No extra Text. "
    "Subject: "
)

# Stil til Jobs (Brede splash screens)
STYLE_SPLASH = (
    "Wide horizontal pixel art illustration, Windows 95 splash screen style. "
    "Retro computing interface, 256 color VGA palette, dithered textures, low resolution CRT monitor effect. No extra Text. "
    "Scene: "
)

# ----------------------------------------------

# INDSTILLINGER
PROJECT_ID = "bobenddk-personal"
LOCATION = "global"  # Gemini models are typically available here

def init_ai():
    print(f"Initializing Google Gen AI Client in {LOCATION}...")
    # Initialize the new Gen AI Client (Vertex AI backend)
    client = genai.Client(
        vertexai=True,
        project=PROJECT_ID,
        location=LOCATION
    )
    return client

def generate_image(client, full_prompt, output_filename, aspect_ratio="1:1"):
    print(f"Generating: {output_filename}...")

    # Append aspect ratio to prompt as a directive (Gemini 3 follows instructions well)
    prompt_text = f"{full_prompt} Aspect Ratio: {aspect_ratio}"

    try:
        response = client.models.generate_content(
            model="gemini-3-pro-image-preview",
            contents=[
                types.Content(
                    role="user",
                    parts=[
                        types.Part.from_text(text=prompt_text)
                    ]
                )
            ],
            config=types.GenerateContentConfig(
                temperature=1,
                response_modalities=["IMAGE"], # Request image output
                safety_settings=[
                    types.SafetySetting(
                        category="HARM_CATEGORY_HATE_SPEECH",
                        threshold="OFF"
                    ),
                    types.SafetySetting(
                        category="HARM_CATEGORY_DANGEROUS_CONTENT",
                        threshold="OFF"
                    ),
                    types.SafetySetting(
                        category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
                        threshold="OFF"
                    ),
                    types.SafetySetting(
                        category="HARM_CATEGORY_HARASSMENT",
                        threshold="OFF"
                    )
                ]
            )
        )

        # Extract and save image from response
        image_saved = False
        if response.candidates:
            for part in response.candidates[0].content.parts:
                if part.inline_data:
                    # Handle data (base64 string or bytes)
                    data = part.inline_data.data
                    if isinstance(data, str):
                        img_data = base64.b64decode(data)
                    else:
                        img_data = data

                    with open(output_filename, "wb") as f:
                        f.write(img_data)
                    print(f"  Saved to {output_filename}")
                    image_saved = True
                    break

        if not image_saved:
            print(f"  No image found in response for {output_filename}")

    except Exception as e:
        print(f"  ERROR generating {output_filename}: {e}")
        time.sleep(5)

def main():
    # Mappe til output
    global OUTPUT_DIR
    OUTPUT_DIR = "generated_images"
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        print(f"Created directory: {OUTPUT_DIR}")

    try:
        with open('prompts.json', 'r') as f:
            data = json.load(f)
    except FileNotFoundError:
        print("Error: prompts.json not found! Please ensure the JSON file is in the same directory.")
        return

    client = init_ai()

    # Helper function to process a list
    def process_list(category_name, item_list, style_prefix, id_key, file_prefix, ar="1:1"):
        for item in tqdm(item_list, desc=category_name):
            # Sætter Style + Beskrivelse sammen her
            final_prompt = style_prefix + item['desc']

            # Håndter ID eller Titel for filnavn
            item_id = item.get(id_key, "unknown").lower().replace(" ", "_")
            filename = os.path.join(OUTPUT_DIR, f"{file_prefix}_{item_id}.png")

            if not os.path.exists(filename):
                generate_image(client, final_prompt, filename, aspect_ratio=ar)
                time.sleep(2) # Pause for at undgå rate limits

    # 1. Projects (Icons)
    process_list("Projects", data.get("projects", []), STYLE_ICON, "id", "project")

    # 2. Meta Upgrades (Icons)
    process_list("Meta Upgrades", data.get("meta_upgrades", []), STYLE_ICON, "id", "meta")

    # 3. Upgrades (Icons)
    process_list("Upgrades", data.get("upgrades", []), STYLE_ICON, "id", "upgrade")

    # 4. Jobs (Splash Screens - 16:9)
    process_list("Jobs", data.get("jobs", []), STYLE_SPLASH, "title", "job", ar="16:9")

    print("\nDone! All images processed.")

In [9]:
main()

Created directory: generated_images
Initializing Google Gen AI Client in global...


Projects:   0%|          | 0/5 [00:00<?, ?it/s]

Generating: generated_images/project_synergy_summit.png...
  Saved to generated_images/project_synergy_summit.png
Generating: generated_images/project_rebranding.png...
  Saved to generated_images/project_rebranding.png
Generating: generated_images/project_offshore_accounts.png...
  Saved to generated_images/project_offshore_accounts.png
Generating: generated_images/project_hostile_takeover.png...
  Saved to generated_images/project_hostile_takeover.png
Generating: generated_images/project_occult_ritual.png...
  Saved to generated_images/project_occult_ritual.png


Meta Upgrades:   0%|          | 0/5 [00:00<?, ?it/s]

Generating: generated_images/meta_insider_trading.png...
  Saved to generated_images/meta_insider_trading.png
Generating: generated_images/meta_golden_parachute.png...
  Saved to generated_images/meta_golden_parachute.png
Generating: generated_images/meta_nepotism.png...
  Saved to generated_images/meta_nepotism.png
Generating: generated_images/meta_blood_pact.png...
  Saved to generated_images/meta_blood_pact.png
Generating: generated_images/meta_void_investment.png...
  Saved to generated_images/meta_void_investment.png


Upgrades:   0%|          | 0/11 [00:00<?, ?it/s]

Generating: generated_images/upgrade_stapler.png...
  Saved to generated_images/upgrade_stapler.png
Generating: generated_images/upgrade_coffee.png...
  Saved to generated_images/upgrade_coffee.png
Generating: generated_images/upgrade_intern.png...
  Saved to generated_images/upgrade_intern.png
Generating: generated_images/upgrade_copier.png...
  Saved to generated_images/upgrade_copier.png
Generating: generated_images/upgrade_computer.png...
  Saved to generated_images/upgrade_computer.png
Generating: generated_images/upgrade_server.png...
  Saved to generated_images/upgrade_server.png
Generating: generated_images/upgrade_algorithm.png...
  Saved to generated_images/upgrade_algorithm.png
Generating: generated_images/upgrade_ai_manager.png...
  Saved to generated_images/upgrade_ai_manager.png
Generating: generated_images/upgrade_neural_link.png...
  Saved to generated_images/upgrade_neural_link.png
Generating: generated_images/upgrade_blood_ink.png...
  Saved to generated_images/upgrad

Jobs:   0%|          | 0/11 [00:00<?, ?it/s]

Generating: generated_images/job_intern.png...
  Saved to generated_images/job_intern.png
Generating: generated_images/job_mailroom_clerk.png...
  Saved to generated_images/job_mailroom_clerk.png
Generating: generated_images/job_junior_associate.png...
  Saved to generated_images/job_junior_associate.png
Generating: generated_images/job_middle_manager.png...
  Saved to generated_images/job_middle_manager.png
Generating: generated_images/job_senior_vp.png...
  Saved to generated_images/job_senior_vp.png
Generating: generated_images/job_ceo.png...
  Saved to generated_images/job_ceo.png
Generating: generated_images/job_board_member.png...
  Saved to generated_images/job_board_member.png
Generating: generated_images/job_chairman.png...
  Saved to generated_images/job_chairman.png
Generating: generated_images/job_shadow_director.png...
  Saved to generated_images/job_shadow_director.png
Generating: generated_images/job_grand_architect.png...
  Saved to generated_images/job_grand_architect.

In [10]:
# prompt: zip all the generated images

!zip -r generated_images.zip generated_images


  adding: generated_images/ (stored 0%)
  adding: generated_images/meta_golden_parachute.png (deflated 0%)
  adding: generated_images/upgrade_intern.png (deflated 0%)
  adding: generated_images/job_mailroom_clerk.png (deflated 0%)
  adding: generated_images/project_hostile_takeover.png (deflated 0%)
  adding: generated_images/upgrade_ai_manager.png (deflated 0%)
  adding: generated_images/project_synergy_summit.png (deflated 0%)
  adding: generated_images/job_junior_associate.png (deflated 0%)
  adding: generated_images/meta_void_investment.png (deflated 0%)
  adding: generated_images/project_offshore_accounts.png (deflated 0%)
  adding: generated_images/meta_insider_trading.png (deflated 0%)
  adding: generated_images/job_middle_manager.png (deflated 0%)
  adding: generated_images/job_shadow_director.png (deflated 0%)
  adding: generated_images/upgrade_copier.png (deflated 0%)
  adding: generated_images/job_chairman.png (deflated 0%)
  adding: generated_images/job_ceo.png (deflated 0%